In [1]:
import torch 
from models import Generator, Discriminator, Encoder
from dataset import get_data_loader, cifar10loader, mnistloader

In [2]:
train_loader, _ = cifar10loader(batch_size=128)#get_data_loader()
encoder= Encoder()
generator = Generator()
discriminator = Discriminator()

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
encoder?

In [3]:
for x, _ in train_loader:
    
    
    print(x.size())
    mu_z, logvar_z, z_samples = encoder(x)
    print(f"mu_z : {mu_z.size()}")
    print(f"logvar_z : {logvar_z.size()}")
    print(f"z_sample : {z_samples.size()}")

    x_gen = generator(mu_z.view(1, mu_z.size(0), mu_z.size(1), 1))
    print(f" x_gen size {x_gen.size()}")
    # proba_x, dis_l = discriminator(x_gen)
    # print(f"proba_x: {proba_x.size()}")
    # print(f"dis_l size: {dis_l.size()}")
    break

torch.Size([128, 3, 64, 64])
z shape torch.Size([128, 5184])
mu size: torch.Size([128, 2592])
log_var size : torch.Size([128, 2592])
epsilon size: torch.Size([128, 2592])
mu_z : torch.Size([128, 2592])
logvar_z : torch.Size([128, 2592])
z_sample : torch.Size([128, 2592])
input size gen : torch.Size([1, 128, 2592, 1])
output size gen : torch.Size([1, 3, 10396, 32])


RuntimeError: The size of tensor a (10396) must match the size of tensor b (32) at non-singleton dimension 2

In [ ]:
mu_z.unsqueeze().size()

In [ ]:
mu_z.size()

In [ ]:
x_true = x.float()
print(f"x_true size : {x_true.size()}")
stack = encoder.encStack(x_true)
stack.size()
stack = stack.view(stack.size(0), -1)
lattent_dim = stack.size(1)//2

z=stack
mu, log_var = z[:, :lattent_dim], z[:,lattent_dim:]
std = torch.exp(0.5* log_var)
print(f"size z : {z.size()}")
epsilon = torch.randn_like(std)
sample = mu + epsilon*std
# sampling latent space corresponding to real data
# sample  = encoder.reparametrize(x_true)

In [ ]:
lattent_dim

In [ ]:
mu.size()

In [ ]:
sample.size()

In [ ]:
sample.view(x_true.size(0), lattent_dim, 1, 1).size()

In [ ]:
stack = stack.view(stack.size(0), -1)
stack.size()
# mu, logvar, sample = encoder.reparametrize(stack)

In [ ]:
x_true = x.float()
print(f"x_true size : {x_true.size()}")
# sampling latent space corresponding to real data
mu_z, logvar_z ,z_true,  = encoder(x_true)
# print(f"z_true : {z_true.size()}")
# KL divergence of the prio
loss_prior =  -0.5 * torch.sum(1 + logvar_z - mu_z.pow(2) - logvar_z.exp())
#  sampling image corresponding to real latent variable 
x_gen = generator(z_true)
print(f"x_gen size : {x_gen.size()}")

In [3]:

import torch 
from train import Trainer
from models import Discriminator, Encoder, Generator
from dataset import get_data_loader, cifar10loader

device = torch.device("cuda")

# dataloader= get_data_loader()
trainloader, testloader  = cifar10loader(batch_size=64)
vae_gan = Trainer(trainloader, Generator, Encoder, Discriminator)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
epochs = 300
for epoch in range(1, epochs + 1):

    vae_gan.train(epoch)
    # vae_gan.test(epoch)


Epoch:1/Iter:0, Total Loss: 8.964813, enc_loss : 8.961025, dec_loss: 0.001899, dis_loss: 0.001889, LK_D: 8.959952
Epoch:1/Iter:50, Total Loss: 0.061066, enc_loss : 42.782119, dec_loss: 0.152311, dis_loss: 0.002422, LK_D: 0.055123
Epoch:1/Iter:100, Total Loss: 0.040226, enc_loss : 44.586240, dec_loss: 0.268242, dis_loss: 0.002239, LK_D: 0.034714
Epoch:1/Iter:150, Total Loss: 0.024327, enc_loss : 45.746495, dec_loss: 0.371064, dis_loss: 0.001967, LK_D: 0.019450
Epoch:1/Iter:200, Total Loss: 0.020208, enc_loss : 46.570328, dec_loss: 0.465047, dis_loss: 0.001816, LK_D: 0.015714
Epoch:1/Iter:250, Total Loss: 0.013854, enc_loss : 47.196710, dec_loss: 0.556144, dis_loss: 0.001801, LK_D: 0.009448
Epoch:1/Iter:300, Total Loss: 0.013362, enc_loss : 47.712836, dec_loss: 0.646074, dis_loss: 0.001790, LK_D: 0.009036
Epoch:1/Iter:350, Total Loss: 0.012168, enc_loss : 48.174009, dec_loss: 0.735595, dis_loss: 0.001777, LK_D: 0.007904
Epoch:2/Iter:0, Total Loss: 8.661384, enc_loss : 8.657569, dec_loss:

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))